In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from collections import defaultdict
import numpy as np
import cv2
import glob
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score



folders = {
    'daisy': r'/content/drive/MyDrive/flower_photos/daisy/*.jpg',
    'dandelion': r'/content/drive/MyDrive/flower_photos/dandelion/*.jpg',
    'roses': r'/content/drive/MyDrive/flower_photos/roses/*.jpg',
    'sunflowers': r'/content/drive/MyDrive/flower_photos/sunflowers/*.jpg',
    'tulips': r'/content/drive/MyDrive/flower_photos/tulips/*.jpg'
}


images = []
labels = []
label_mapping = {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

for label, folder_path in folders.items():
    for img_path in glob.glob(folder_path):
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
            labels.append(label_mapping[label])


def extract_features(image):
    image = cv2.resize(image, (128, 128))

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    color_features = cv2.mean(hsv)[:3]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_features = cv2.mean(edges)[:1]

    return np.concatenate([color_features, edge_features])

print("Extracting features...")
features = np.array([extract_features(img) for img in images])

n_clusters = 3000
print("Quantizing features...")
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
quantized_features = kmeans.fit_transform(features)

states = np.argmin(quantized_features, axis=1)

group_mapping = {
    'daisy': 0,
    'dandelion': 0,
    'roses': 1,
    'sunflowers': 2,
    'tulips': 3
}

num_actions = len(label_mapping)
learning_rate = 0.1
discount_factor = 0.95
epsilon_start = 1
epsilon_end = 0.01
epsilon_decay = 0.995
episodes = 10000

q_tables = {
    0: defaultdict(lambda: np.zeros(num_actions)),
    1: defaultdict(lambda: np.zeros(num_actions)),
    2: defaultdict(lambda: np.zeros(num_actions)),
    3: defaultdict(lambda: np.zeros(num_actions))
}

print("Starting training...")
epsilon = epsilon_start
for episode in range(episodes):
    for state, true_label in zip(states, labels):

        group = group_mapping[list(label_mapping.keys())[true_label]]
        q_table = q_tables[group]

        if np.random.rand() < epsilon:
            action = np.random.randint(num_actions)
        else:
            action = np.argmax(q_table[state])

        if action == true_label:
            reward = 2
        elif abs(action - true_label) == 1:
            reward = 0.5
        else:
            reward = -1

        next_state = state

        next_max = np.max(q_table[next_state])
        q_table[state][action] += learning_rate * (reward + discount_factor * next_max - q_table[state][action])

    epsilon = max(epsilon_end, epsilon * epsilon_decay)

    if episode % 1000 == 0:
        print(f"Completed episode {episode}, epsilon: {epsilon:.4f}")

print("Testing...")
y_pred = []
for state, true_label in zip(states, labels):
    group = group_mapping[list(label_mapping.keys())[true_label]]
    q_table = q_tables[group]
    action = np.argmax(q_table[state])
    y_pred.append(action)

accuracy = accuracy_score(labels, y_pred)
precision = precision_score(labels, y_pred, average='weighted')

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')

for group, q_table in q_tables.items():
    print(f'Number of unique states for group {group}: {len(q_table)}')
    print(f'Average Q values for group {group}: {np.mean([np.max(q) for q in q_table.values()])}')


Extracting features...
Quantizing features...
Starting training...
Completed episode 0, epsilon: 0.9950
Completed episode 1000, epsilon: 0.0100
Completed episode 2000, epsilon: 0.0100
Completed episode 3000, epsilon: 0.0100
Completed episode 4000, epsilon: 0.0100
Completed episode 5000, epsilon: 0.0100
Completed episode 6000, epsilon: 0.0100
Completed episode 7000, epsilon: 0.0100
Completed episode 8000, epsilon: 0.0100
Completed episode 9000, epsilon: 0.0100
Testing...
Accuracy: 98.01%
Precision: 98.02%
Number of unique states for group 0: 1371
Average Q values for group 0: 39.24625444293572
Number of unique states for group 1: 623
Average Q values for group 1: 39.9999999999992
Number of unique states for group 2: 625
Average Q values for group 2: 39.9999999999992
Number of unique states for group 3: 759
Average Q values for group 3: 39.9999999999992


In [3]:
import numpy as np
from collections import defaultdict
import cv2
import glob
from sklearn.metrics import accuracy_score, precision_score


folders = {
    'daisy': r'/content/drive/MyDrive/flower_photos/daisy/*.jpg',
    'dandelion': r'/content/drive/MyDrive/flower_photos/dandelion/*.jpg',
    'roses': r'/content/drive/MyDrive/flower_photos/roses/*.jpg',
    'sunflowers': r'/content/drive/MyDrive/flower_photos/sunflowers/*.jpg',
    'tulips': r'/content/drive/MyDrive/flower_photos/tulips/*.jpg'
}


def load_images(folder_path, label):
    images = []
    for img_path in glob.glob(folder_path):
        img = cv2.imread(img_path)
        if img is not None:
            img_resized = cv2.resize(img, (250, 250))
            hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
            feature_vector = hsv.flatten()[:70]
            images.append((feature_vector, label))
    return images

data = []
label_mapping = {label: idx for idx, label in enumerate(folders.keys())}
for label, folder in folders.items():
    data.extend(load_images(folder, label_mapping[label]))


features, labels = zip(*data)
features = np.array(features)
labels = np.array(labels)



num_states = features.shape[1]
num_actions = len(label_mapping)
learning_rate = 0.1
discount_factor = 0.95
epsilon_start = 0.9
epsilon_end = 0.1
epsilon_decay = 0.995
episodes = 1000


q_table = defaultdict(lambda: np.zeros(num_actions))


def choose_action(state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.randint(num_actions)
    else:
        return np.argmax(q_table[state])


def update_q_table(state, action, reward, next_state):
    best_next_action = np.argmax(q_table[next_state])
    td_target = reward + discount_factor * q_table[next_state][best_next_action]
    td_error = td_target - q_table[state][action]
    q_table[state][action] += learning_rate * td_error


epsilon = epsilon_start
for episode in range(episodes):
    for feature, true_label in zip(features, labels):
        state = tuple(np.round(feature, 2))
        action = choose_action(state, epsilon)
        reward = 2 if action == true_label else -1
        next_state = state
        update_q_table(state, action, reward, next_state)


    epsilon = max(epsilon_end, epsilon * epsilon_decay)


    if episode % 100 == 0:
        print(f"Episode {episode}: epsilon = {epsilon:.3f}")


y_pred = []
for feature in features:
    state = tuple(np.round(feature, 2))
    action = np.argmax(q_table[state])
    y_pred.append(action)


accuracy = accuracy_score(labels, y_pred)
precision = precision_score(labels, y_pred, average='weighted')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')



Episode 0: epsilon = 0.895
Episode 100: epsilon = 0.542
Episode 200: epsilon = 0.329
Episode 300: epsilon = 0.199
Episode 400: epsilon = 0.121
Episode 500: epsilon = 0.100
Episode 600: epsilon = 0.100
Episode 700: epsilon = 0.100
Episode 800: epsilon = 0.100
Episode 900: epsilon = 0.100
Accuracy: 97.06%
Precision: 97.21%
